# Generating the annotation table from IAW

In this notebook, we will showcase how to create a table that displays annotation data and the corresponding annotated area image by using the Image Annotation Workbench (IAW) API.

To get start, we need to import a few dependencies. We will use OpenCV to process images and crop the annotated areas from the original images.

In [ ]:
import cv2
import numpy as np
import requests
import base64
from IPython.display import display, HTML
import pandas as pd
import json

In the upcoming sections of the code, we will be defining some functions that will come in handy later on.

In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL into OpenCV.

    Parameters
    ----------
    url : str
        The URL of the image to load.

    Returns
    -------
    image : numpy.ndarray
        The image loaded into OpenCV.
    """
    response = requests.get(url)
    image = cv2.imdecode(np.frombuffer(response.content, np.uint8), cv2.IMREAD_COLOR)
    return image


In [ ]:
def crop_image(image, points):
    """
    Crop an image using a polygon selector.

    Parameters
    ----------
    image : numpy.ndarray
        The image to crop.
    points : list
        The points of the polygon selector.

    Returns
    -------
    cropped_image : numpy.ndarray
        The cropped image.
    """
    # Create a mask.
    mask = np.zeros(image.shape, dtype=np.uint8)
    roi_corners = np.array([points], dtype=np.int32)
    channel_count = image.shape[2]
    ignore_mask_color = (255,) * channel_count
    cv2.fillPoly(mask, roi_corners, ignore_mask_color)
    # Apply the mask.
    masked_image = cv2.bitwise_and(image, mask)
    # Crop out the blank area.
    y, x, _ = np.where(masked_image != 0)
    cropped_image = masked_image[np.min(y): np.max(y), np.min(x): np.max(x)]
    return cropped_image


In [ ]:
def image_to_html(image, max_width=None):
    """
    Convert an image to HTML for display in IPython.

    Parameters
    ----------
    image : numpy.ndarray
        The image to convert to HTML.
    max_width : int, optional
        The maximum width of the image in pixels.

    Returns
    -------
    html : str
        The HTML to display the image in IPython.
    """
    retval, buffer = cv2.imencode('.jpg', image)
    jpg_as_text = base64.b64encode(buffer)
    image_width = 'width:100%';
    if max_width is not None:
        image_width = 'max-width:{}px'.format(max_width)
    html = '<img style="{}" src="data:image/jpeg;base64,{}"/>'.format(image_width, jpg_as_text.decode('utf-8'))
    return html


In [ ]:
def extract_polygon_points(selector):
    """
    Extract the value from the 'points' attribute of the SVG polygon selector.

    Parameters
    ----------
    selector : str
        The SVG polygon selector. e.g. <svg><polygon points="137.23291015625,433.1348571777344 396.35723876953125,431.2434387207031 413.3800354003906,461.5061340332031 403.92291259765625,480.4203186035156"></polygon></svg>

    Returns
    -------
    points : list
        The polygon points.
    """
    selector_value = selector.split('"')[1]
    points = []
    for point in selector_value.split(' '):
        x, y = point.split(',')
        points.append([int(float(x)), int(float(y))])
    return points


To use the IAW API, we firstly need to specify the API URL and token. The API token will be required in the `Authorization` header when sending requests.

In [ ]:
# The Base URL of IAW API.
iaw_api_base = 'https://iaw-server.ardc-hdcl-sia-iaw.cloud.edu.au/api'
# API token.
iaw_api_token = '3|dajsYsH0Dx87OXlAWk7T5ZDRJGgCMaV5WJhgKnth32114c9a'

# Set the headers
headers = {
    'Accept': 'application/json',
    'Authorization': f'Bearer {iaw_api_token}'
}

Now we are going to make our first request to the IAW API to list all collections from the authenticated account.

In [ ]:
# Read all collections from the API.
response = requests.get(f'{iaw_api_base}/collections', headers=headers)
print(str(response.status_code) + ' ' + response.reason)
collections = response.json()
print(json.dumps(collections, indent=2))

From the response data, we can see a list of collection objects from IAW. We will use the "Image Gandhara" collection for the demo. We need to take a note of its `id` which is `11` in this case as we need it to construct our following API calls.

With the collection `id`, we will query all image sets from this collection. Note how we use the collection `id` in the request URL.

In [ ]:
# Read all image sets from the collection.
response = requests.get(f'{iaw_api_base}/collections/11/image-sets', headers=headers)
print(str(response.status_code) + ' ' + response.reason)
image_sets = response.json()
print(json.dumps(image_sets, indent=2))

From the response, we can see there are a number of image sets in collection "Image Gandhara". We will only use the image set "Schist Buddha Triad (year 5)" for this demonstration. Again, we need its `id` (`31`) for our following requests.

Next, we will get a list of images from the image set "Schist Buddha Triad (year 5)".

In [ ]:
# Read all images from the image set.
response = requests.get(f'{iaw_api_base}/image-sets/31/images', headers=headers)
print(str(response.status_code) + ' ' + response.reason)
images = response.json()
print(json.dumps(images, indent=2))

From the response, we can see there's only a single image in the image set "Schist Buddha Triad (year 5)". Because all annotations from IAW are based on the original size of images, we will need to load the image in its original size in our code in order to crop the annotated areas. We will use the IIIF Image API to get the basic information about the image.

We can get the IIIF base URL (`iiif_url`) of the image from the response above. Then we will send a request to `info.json` of that image to get the information.

In [ ]:
image_data = images[0]
image_iiif_base_url = image_data['iiif_url']
# Read the image info.
response = requests.get(f'{image_iiif_base_url}/info.json')
print(str(response.status_code) + ' ' + response.reason)
image_info = response.json()
print(json.dumps(image_info, indent=2))

We can then get the original `width` and `height` of the image from the response of `info.json` and load the image in its original size with IIIF Image API.

In [ ]:
image_width = image_info['width']
image_height = image_info['height']
image_url = f'{image_iiif_base_url}/full/{image_width},{image_height}/0/default.jpg'
image = load_image_from_url(image_url)
display(HTML(image_to_html(image)))

Now we are going to query all annotation sets from the image set "Schist Buddha Triad (year 5)".

In [ ]:
# Read all annotation sets from the image set.
response = requests.get(f'{iaw_api_base}/image-sets/31/annotation-sets', headers=headers)
print(str(response.status_code) + ' ' + response.reason)
annotation_sets = response.json()
print(json.dumps(annotation_sets, indent=2))

We can see that there's only one annotation set from the response. Therefore, we call the API to get all annotations from this annotation set.

In [ ]:
# Read all annotations from the annotation set.
response = requests.get(f'{iaw_api_base}/annotation-sets/26/annotations', headers=headers)
print(str(response.status_code) + ' ' + response.reason)
annotations = response.json()
print(json.dumps(annotations, indent=2))

We now have the data of all annotations. We will create a table to display our annotations. The table will have 5 columns:

- Image: The image of the annotated area.
- Title: The annotation title.
- Tags: The tags applied to the annotaion.
- Notes: Notes of the annotation.
- Line Color: The line color of the annotation.

<div class="alert alert-block alert-info">
<strong>Note</strong>: Due to the size of the image and the number of annotations to process, the following code block may take a minute to finish. Please wait patiently to see the final output.
</div>

In [ ]:
table_data = {
    'Image': [],
    'Title': [],
    'Tags': [],
    'Notes': [],
    'Line Color': [],
}
for annotation in annotations:
    # Crop annotation area.
    selector = annotation['target']['selector']['value']
    if '<polygon' in selector:
        points = extract_polygon_points(selector)
        cropped_image = crop_image(image, points)
        table_data['Image'].append(image_to_html(cropped_image, 250))
    else:
        table_data['Image'].append('')
    fields = annotation['fields']
    # Annotation title.
    if 'title' in fields:
        table_data['Title'].append('<br>'.join(fields['title']['en']['values']))
    else:
        table_data['Title'].append('')
    # Annotation tags.
    if 'tag' in fields:
        table_data['Tags'].append('<br>'.join([item['term_label'] for item in fields['tag']['en']['values']]))
    else:
        table_data['Tags'].append('')
    # Annotation notes.
    if 'note' in fields:
        table_data['Notes'].append('<br>'.join(fields['note']['en']['values']))
    else:
        table_data['Notes'].append('')
    # Line color.
    if 'line_color' in fields:
        table_data['Line Color'].append('<br>'.join(fields['line_color']['en']['values']))
    else:
        table_data['Line Color'].append('')

# Create a DataFrame from the dictionary
df = pd.DataFrame(table_data)

# Convert the DataFrame to a HTML table
html = df.to_html(escape=False, index=False)

# Set content alignment to left
html = html.replace('<th>', '<th style="text-align: left;">')
html = html.replace('<td>', '<td style="text-align: left;">')

# Display the HTML table
display(HTML(html))
    
